In [1]:
!git clone https://github.com/boudinfl/pke.git
!pip install --upgrade pip setuptools
!pip install pke
!pip install spacy-transformers
!python setup.py install
!pip install spacy
!pip install tweetnlp
!pip install ProfanityDetector

Cloning into 'pke'...
remote: Enumerating objects: 2489, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 2489 (delta 42), reused 58 (delta 33), pack-reused 2406
Receiving objects: 100% (2489/2489), 85.46 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (1538/1538), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pke (from versions: none)
ERROR: No matching distribution found for pke
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━

In [2]:
cd pke

/content/pke


In [3]:
import pke
import sys
import os
import pandas as pd
import spacy
import tweetnlp
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from ProfanityDetector import detector
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import nltk
import re
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nlp = spacy.load("en_core_web_sm")
propnames = tweetnlp.load_model('ner')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [6]:
tweetsFile = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/topics_preprocessed.csv")
tweets_topic0 = []
for i in range(len(tweetsFile.Tweet)):
    if tweetsFile.Topic[i] == 0:
        tweets_topic0.append(tweetsFile.Tweet[i])
print(len(tweets_topic0))

106


In [7]:
tweetsFile = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/todayFinish.csv")
Preprocessed_topic0 = []
Main_topic0 = []
for i in range(len(tweetsFile.Preprocessed)):
    if tweetsFile.Topic[i] == 0:
        Preprocessed_topic0.append(tweetsFile.Preprocessed[i])
        Main_topic0.append(tweetsFile.Tweet[i])

print(len(Preprocessed_topic0))
for i in range(len(Preprocessed_topic0)):
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("de zerbi", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("dezerbi", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("zerbi", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace(" de ", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("roberto", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("robert", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("francesco", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("rdz", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("brighton", "")
    Preprocessed_topic0[i] = Preprocessed_topic0[i].replace("bha", "")

106


In [8]:
def find_propNounPre(tweet, proc):
    tweet = re.sub(r"/", " / ", tweet)
    tweet = re.sub(r",", " , ", tweet)
    t = propnames.ner(tweet)
    propn_tweet = []
    for i in t:
        if i['type'] == 'person':
            e = i['entity'].strip()
            if e not in propn_tweet:
                if len(e.split(" ")) > 1 and e[len(e.split(" "))-1] in proc:
                    propn_tweet.append(e[len(e.split(" "))-1].lower())
                elif e.lower() in proc:
                    propn_tweet.append(e.lower())
    return len(propn_tweet)
propNouns_pre = []
for i in range(len(Main_topic0)):
    pN = find_propNounPre(Main_topic0[i], Preprocessed_topic0[i])
    propNouns_pre.append(pN)

zeros = 0
ones = 0
twos = 0
threes = 0
els = 0
for i in range(len(propNouns_pre)):
    if propNouns_pre[i] == 0:
        zeros += 1
    elif propNouns_pre[i] == 1:
        ones += 1
    elif propNouns_pre[i] == 2:
        twos += 1
    elif propNouns_pre[i] == 3:
        threes += 1
    else:
        els += 1

print(zeros)
print(ones)
print(twos)
print(threes)
print(els)

33
50
13
5
5


In [9]:
def multipartite_rank_extractor(text):
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(text, language='en')
    pos = {'NOUN', 'ADJ', 'VERB'}
    extractor.candidate_selection(pos=pos)
    extractor.candidate_weighting(alpha=1.1, threshold=0.74, method='average')
    keyphrases = extractor.get_n_best(n=10)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results

In [10]:
keywords_allTweet = []
for tweet in tweets_topic0:
    kt = []
    keywords_temp = multipartite_rank_extractor(tweet.lower())
    for key in keywords_temp:
        s = key.split(' ')
        if len(s) == 2:
              for token in nlp(s[1]):
                  if token.pos_ == 'NOUN':
                      for token in nlp(s[0]):
                          if token.pos_ == 'VERB' or token.pos_ == 'ADJ':
                              kt.append(s[0])
                              kt.append(s[1])
        elif len(s) == 1:
            kt.append(key)
    keywords_allTweet.append(kt)

In [11]:
print(len(keywords_allTweet))
print(keywords_allTweet)

106
[['years', 'exclusive', 'club', 'attacking', 'philosophy', 'helm', 'way'], ['expectations', 'lid'], ['absorb', 'tempted', 'trying', 'episode', 'press', 'style', 'loved'], ['fulfil', 'trossard', 'potential', 'happened'], ['club', 'addition', 'ripped', 'heart', 'rebuild', 'pressure', 'kind', 'chaps'], ['trossard', 'groß', 'best', 'attacker', 'macallister', 'supportive', 'pick'], ['watching', 'evolution', 'interesting', 'observe', 'bha', 'going', 'analyze', 'priority'], ['think', 'permanent', 'option', 'trossard', 'making', 'shift', 'depends', 'temporary', 'solution', 'big', 'ones', 'adingra', 'options'], ['vows', 'attacking', 'force', 'add', 'liverpool', 'new', 'boss', 'afraid'], ['nfo', 'increase', 'appeal', 'sure', 'bha', 'fan', 'use', 'think', 'picks', 'rotate'], ['attacking'], ['hat-trick', 'problem', 'challenged', 'liverpo', 'new', 'boss', 'potter'], ['winning', 'ball', 'pressure', 'controlling', 'tough', 'opponents', 'terrorised', 'liverpool', 'niced'], ['playing', 'attracting'

In [12]:
def find_propNoun(tweet):
    tweet = re.sub(r"/", " / ", tweet)
    tweet = re.sub(r",", " , ", tweet)
    t = propnames.ner(tweet)
    propn_tweet = []
    group_nouns = []
    for i in t:
        if i['type'] == 'person':
            e = i['entity'].strip()
            if e not in propn_tweet:
                propn_tweet.append(e.lower())
        elif i['type'] == 'group':
            e = i['entity'].strip()
            if e not in group_nouns:
                group_nouns.append(e.lower())
    return propn_tweet, group_nouns

# add propNouns of groups
for i in range(len(tweets_topic0)):
    pN , gN = find_propNoun(tweets_topic0[i])
    for item in gN:
        if item not in keywords_allTweet[i] and len(item.split(" ")) < 2:
            keywords_allTweet[i].append(item)

print(len(keywords_allTweet))
print(keywords_allTweet)

106
[['years', 'exclusive', 'club', 'attacking', 'philosophy', 'helm', 'way', 'brighton', 'albion'], ['expectations', 'lid'], ['absorb', 'tempted', 'trying', 'episode', 'press', 'style', 'loved'], ['fulfil', 'trossard', 'potential', 'happened', 'liverpool', 'brighton'], ['club', 'addition', 'ripped', 'heart', 'rebuild', 'pressure', 'kind', 'chaps'], ['trossard', 'groß', 'best', 'attacker', 'macallister', 'supportive', 'pick'], ['watching', 'evolution', 'interesting', 'observe', 'bha', 'going', 'analyze', 'priority'], ['think', 'permanent', 'option', 'trossard', 'making', 'shift', 'depends', 'temporary', 'solution', 'big', 'ones', 'adingra', 'options'], ['vows', 'attacking', 'force', 'add', 'liverpool', 'new', 'boss', 'afraid', 'brighton'], ['nfo', 'increase', 'appeal', 'sure', 'bha', 'fan', 'use', 'think', 'picks', 'rotate'], ['attacking', 'brighton', 'forest'], ['hat-trick', 'problem', 'challenged', 'liverpo', 'new', 'boss', 'potter', "brighton's"], ['winning', 'ball', 'pressure', 'co

In [13]:
propNouns = []
for tweet1 in tweets_topic0:
    pN, gN = find_propNoun(tweet1)
    for item in pN:
        if item not in propNouns:
            propNouns.append(item)

# delete propNouns of person
prop = ' '.join(propNouns)
for i in range(len(keywords_allTweet)):
    for token in keywords_allTweet[i]:
        if token in prop:
            keywords_allTweet[i].remove(token)

print(len(keywords_allTweet))
print(keywords_allTweet)

106
[['years', 'exclusive', 'club', 'attacking', 'philosophy', 'helm', 'way', 'albion'], ['expectations', 'lid'], ['absorb', 'tempted', 'trying', 'episode', 'press', 'style', 'loved'], ['fulfil', 'potential', 'happened', 'liverpool'], ['club', 'addition', 'ripped', 'heart', 'rebuild', 'pressure', 'kind', 'chaps'], ['groß', 'best', 'attacker', 'supportive', 'pick'], ['watching', 'evolution', 'interesting', 'observe', 'bha', 'going', 'analyze', 'priority'], ['think', 'permanent', 'option', 'making', 'shift', 'depends', 'temporary', 'solution', 'big', 'ones', 'options'], ['vows', 'attacking', 'force', 'liverpool', 'new', 'boss', 'afraid'], ['nfo', 'increase', 'appeal', 'sure', 'bha', 'fan', 'use', 'think', 'picks', 'rotate'], ['attacking', 'forest'], ['hat-trick', 'problem', 'challenged', 'liverpo', 'new', 'boss', "brighton's"], ['winning', 'pressure', 'controlling', 'tough', 'opponents', 'terrorised', 'liverpool', 'niced'], ['playing', 'attracting', 'pressure', 'rock', 'games', 'lot', 'g

In [14]:
lemmatizer = WordNetLemmatizer()

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

lemma = []
for tweet in keywords_allTweet:
    pos_tagged = nltk.pos_tag(tweet)
    wordnet_tagged = [*map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged)]
    for word, tag in wordnet_tagged:
        if tag is None:
            lemma.append(word.lower())
        else:
            lemma.append(lemmatizer.lemmatize(word, tag))  

print(len(lemma))
print(lemma)

328
['year', 'exclusive', 'club', 'attack', 'philosophy', 'helm', 'way', 'albion', 'expectation', 'lid', 'absorb', 'tempt', 'try', 'episode', 'press', 'style', 'love', 'fulfil', 'potential', 'happen', 'liverpool', 'club', 'addition', 'rip', 'heart', 'rebuild', 'pressure', 'kind', 'chap', 'groß', 'best', 'attacker', 'supportive', 'pick', 'watch', 'evolution', 'interest', 'observe', 'bha', 'go', 'analyze', 'priority', 'think', 'permanent', 'option', 'make', 'shift', 'depends', 'temporary', 'solution', 'big', 'one', 'option', 'vow', 'attack', 'force', 'liverpool', 'new', 'bos', 'afraid', 'nfo', 'increase', 'appeal', 'sure', 'bha', 'fan', 'use', 'think', 'pick', 'rotate', 'attack', 'forest', 'hat-trick', 'problem', 'challenge', 'liverpo', 'new', 'bos', "brighton's", 'win', 'pressure', 'control', 'tough', 'opponent', 'terrorise', 'liverpool', 'niced', 'play', 'attract', 'pressure', 'rock', 'game', 'lot', 'go', 'brentford', 'effective', 'teach', 'enjoy', 'discussion', 'current', 'gw9', 'expe

In [15]:
from nltk.corpus import words

In [ ]:
unique_keywords = []
for word in lemma:
    if (word not in unique_keywords) and word in words.words():
        unique_keywords.append(word)
print(len(unique_keywords))
print(unique_keywords)

In [16]:
def find_propNounPre(tweet, proc):
    tweet = re.sub(r"/", " / ", tweet)
    tweet = re.sub(r",", " , ", tweet)
    t = propnames.ner(tweet)
    propn_tweet = []
    for i in t:
        if i['type'] == 'person':
            e = i['entity'].strip()
            if e not in propn_tweet:
                if len(e.split(" ")) > 1 and e[len(e.split(" "))-1] in proc:
                    propn_tweet.append(e[len(e.split(" "))-1].lower()+e.lower())
                elif e.lower() in proc:
                    propn_tweet.append(e.lower())
    return propn_tweet
propNouns_pre = []
for i in range(len(Main_topic0)):
    pN = find_propNounPre(Main_topic0[i], Preprocessed_topic0[i])
    propNouns_pre.append(pN)
tuplesList = []
for i in range(len(propNouns_pre)):
    for j in range(len(propNouns_pre[i])):
        if len(propNouns_pre[i]) == 0:
            if ("", "") not in tuplesList:
                tuplesList.append(("", ""))
        else:
            if j != len(propNouns_pre[i]) - 1:
                if (propNouns_pre[i][j], propNouns_pre[i][j+1]) not in tuplesList and (propNouns_pre[i][j+1], propNouns_pre[i][j]) not in tuplesList:
                    tuplesList.append((propNouns_pre[i][j], propNouns_pre[i][j+1]))
            if ("", propNouns_pre[i][j]) not in tuplesList:
                tuplesList.append(("", propNouns_pre[i][j]))

In [17]:
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []
list90 = []
item1 = ("de zerbi", "")
item2 = ("de zerbi", "mitoma")
item3 = ("de zerbi", "trossard")
item4 = ("de zerbi", "macallister")
item5 = ("de zerbi", "lamptey")
item6 = ("de zerbi", "estupinan")
item7 = ("mitoma", "trossard")
item8 = ("trossard", "macallister")
item9 = ("mitoma", "lamptey")
for item in Main_topic0:
    if item.lower().find(item1[0]) != -1 and item.lower().find(item1[1]) != -1:
        list1.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item2[0]) != -1 and item.lower().find(item2[1]) != -1:
        list2.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item3[0]) != -1 and item.lower().find(item3[1]) != -1:
        list3.append(Preprocessed_topic0[Main_topic0.index(item)])
        list90.append(Main_topic0[Main_topic0.index(item)])
    if item.lower().find(item4[0]) != -1 and item.lower().find(item4[1]) != -1:
        list4.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item5[0]) != -1 and item.lower().find(item5[1]) != -1:
        list5.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item6[0]) != -1 and item.lower().find(item6[1]) != -1:
        list6.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item7[0]) != -1 and item.lower().find(item7[1]) != -1:
        list7.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item8[0]) != -1 and item.lower().find(item8[1]) != -1:
        list8.append(keywords_allTweet[Main_topic0.index(item)])
    if item.lower().find(item9[0]) != -1 and item.lower().find(item9[1]) != -1:
        list9.append(keywords_allTweet[Main_topic0.index(item)])
print(list1)
print(list2)
print(list3)
print(list4)
print(list5)
print(list6)
print(list7)
print(list8)
print(list9)

[['years', 'exclusive', 'club', 'attacking', 'philosophy', 'helm', 'way', 'albion'], ['expectations', 'lid'], ['absorb', 'tempted', 'trying', 'episode', 'press', 'style', 'loved'], ['fulfil', 'potential', 'happened', 'liverpool'], ['club', 'addition', 'ripped', 'heart', 'rebuild', 'pressure', 'kind', 'chaps'], ['groß', 'best', 'attacker', 'supportive', 'pick'], ['watching', 'evolution', 'interesting', 'observe', 'bha', 'going', 'analyze', 'priority'], ['think', 'permanent', 'option', 'making', 'shift', 'depends', 'temporary', 'solution', 'big', 'ones', 'options'], ['vows', 'attacking', 'force', 'liverpool', 'new', 'boss', 'afraid'], ['nfo', 'increase', 'appeal', 'sure', 'bha', 'fan', 'use', 'think', 'picks', 'rotate'], ['attacking', 'forest'], ['hat-trick', 'problem', 'challenged', 'liverpo', 'new', 'boss', "brighton's"], ['winning', 'pressure', 'controlling', 'tough', 'opponents', 'terrorised', 'liverpool', 'niced'], ['playing', 'attracting', 'pressure', 'rock', 'games', 'lot', 'going

In [18]:
!pip install transformers
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('stsb-roberta-large')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [19]:
# sentences1 = ["I like Python because I can build AI applications", "The cat sits on the ground"]   
# sentences2 = ["I like Python because I can do data analytics", "The cat walks on the sidewalk"]
# list_p = []
# for tweet in list3:
#     temp = ' '.join(tweet)
#     list_p.append(temp)
# encode list of sentences to get their embeddings
embedding1 = model.encode(list3, convert_to_tensor=True)
# embedding2 = model.encode(list3, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding1)

In [46]:
matrix_similarity = []
for i in range(len(list3)):
    temp_similarity = []
    for j in range(len(list3)):
        temp_similarity.append(cosine_scores[i][j].item())
    matrix_similarity.append(temp_similarity)

print(matrix_similarity)

[[1.0, 0.33150312304496765, 0.3264240026473999, 0.4555685520172119, 0.3687978982925415, 0.5222668051719666, 0.3838956356048584, 0.2695590853691101, 0.6281946301460266, 0.3687557578086853, 0.27250415086746216, 0.622667670249939, 0.4716542363166809, 0.3303218483924866, 0.25722038745880127, 0.34664660692214966, 0.2859557569026947, 0.44879814982414246, 0.43264463543891907, 0.33462849259376526], [0.33150312304496765, 0.9999999403953552, 0.22450804710388184, 0.4438836872577667, 0.2806205749511719, 0.2798323631286621, 0.5975612998008728, 0.42481493949890137, 0.47080788016319275, 0.1692272573709488, 0.26374197006225586, 0.35219812393188477, 0.5182602405548096, 0.2895832061767578, 0.27505820989608765, 0.1926260143518448, 0.15873250365257263, 0.49471962451934814, 0.3846707344055176, 0.2433832883834839], [0.3264240026473999, 0.22450804710388184, 1.000000238418579, 0.3788803219795227, 0.4421271085739136, 0.6132142543792725, 0.48420238494873047, 0.3436878025531769, 0.4335640072822571, 0.59053719043

In [55]:
def find_max(list_pr, i, indexes):
    max = (0, i, 0)
    for j in range(len(list_pr)):
        if list_pr[i][j] > max[0]:
            if j not in indexes:
                max = (list_pr[i][j], i , j)
    return max

concated_string = ''
concated_indexes = [5]
for item in range(len(matrix_similarity)-1):
    print("concated_indexes: ", concated_indexes)
    score, i, j = find_max(matrix_similarity, concated_indexes[len(concated_indexes)-1], concated_indexes)
    print('score: ', score)
    print("i: ", i)
    print("j: ", j)
    concated_indexes.append(j)
    concated_string += list90[j]
    concated_string += "\n"
    concated_string += "-------"
    print("concated_indexes: ", concated_indexes)
    print('--------------------------------------------------------')

# print(concated_indexes)

concated_indexes:  [5]
score:  0.6772254109382629
i:  5
j:  11
concated_indexes:  [5, 11]
--------------------------------------------------------
concated_indexes:  [5, 11]
score:  0.6492225527763367
i:  11
j:  8
concated_indexes:  [5, 11, 8]
--------------------------------------------------------
concated_indexes:  [5, 11, 8]
score:  0.6281946301460266
i:  8
j:  0
concated_indexes:  [5, 11, 8, 0]
--------------------------------------------------------
concated_indexes:  [5, 11, 8, 0]
score:  0.4716542363166809
i:  0
j:  12
concated_indexes:  [5, 11, 8, 0, 12]
--------------------------------------------------------
concated_indexes:  [5, 11, 8, 0, 12]
score:  0.5524997115135193
i:  12
j:  4
concated_indexes:  [5, 11, 8, 0, 12, 4]
--------------------------------------------------------
concated_indexes:  [5, 11, 8, 0, 12, 4]
score:  0.5451040863990784
i:  4
j:  6
concated_indexes:  [5, 11, 8, 0, 12, 4, 6]
--------------------------------------------------------
concated_indexes:  [

In [56]:
concated_string

"3. Leandro Trossard - MID - BHA - £6.7m\nBagged a hat-trick against Liverpool at Anfield in GW9, registering an xG of 1.79 and 4 big chances, the most out of any player last week \nWill play OOP under new boss De Zerbi as a striker\nShort term fixtures aren’t the best\n-------No Potter, no problem! Leandro Trossard reveals he was challenged by his new boss Roberto De Zerbi to score more goals... so Brighton's hotshot Belgian grabbed a hat-trick against Liverpo...\nvia\n-------A first top-flight hat-trick from a Brighton player for 42 years\nLeandro Trossard joins a very exclusive club. Expect more Albion players to make their way into it with Roberto De Zerbi and his attacking philosophy at the helm\n-------Unfortunately can only pick 2 per category. \nZaha was a buy suggestion from me last week too and his fixture run is great. \nBowens an easy hold but I’d prioritise Maddison, Zaha or even Foden to bring in, personally. \nTrossard has Spurs - chance to wait, watch BHA under De Zerbi